In [ ]:
%matplotlib widget
from pathlib import Path
from shutil import copy

import flammkuchen as fl
import numpy as np
import pandas as pd
from lotr import A_FISH, LotrExperiment, dataset_folders
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
exp.morphed_coords

In [ ]:
np.save(exp.root / "centroid.npy", offs)

In [ ]:
np.load(exp.root / "centroid.npy")

In [ ]:
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    with open(exp.root / "centroid.txt", "r") as f:
        centroids = f.readlines()
    offs = np.array([float(f) for f in centroids])
    transform_mat = np.eye(4)
    transform_mat[:3, 3] = -offs

    np.save(path / "centering_mtx.npy", transform_mat)

In [ ]:
transform_points(exp.coords, trasform_mat)

In [ ]:
def transform_points(pts, mat):
    """Transform array of points using a 4x4 transformation matrix,
    adding 1 column for the offset.
    """

    pts = np.array(pts)
    if len(pts.shape) == 1:
        pts = pts[np.newaxis, :]
    pts = np.insert(pts, pts.shape[1], np.ones(pts.shape[0]), axis=1)
    return (mat @ pts.T).T[:, :3]

In [ ]:
coords_loc = Path("/Volumes/Shared/experiments/E0071_lotr/full_ring/")

In [ ]:
coords_loc.exists()

In [ ]:
for path in tqdm(dataset_folders):
    try:
        src = coords_loc / path.parent.name / path.name / "transformaed_coord2s.h5"
        copy(src, path)
    except:
        print("could not copy ", path.name)

In [ ]:
path = dataset_folders[0]
exp = LotrExperiment(path)

In [ ]:
morph_coords.shape

In [ ]:
morph_coords2.shape

In [ ]:
path.name

In [ ]:
coords_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    coords = exp.coords_um
    cent_coords = coords - np.mean(coords, 0)
    try:
        morph_coords = exp.morphed_coords
        morph_coords2 = fl.load(path / "transformaed_coord2s.h5", "/transformed_coords")
    except OSError:
        morph_coords = np.full(coords.shape, np.nan)
        morph_coords2 = np.full(morph_coords.shape, np.nan)
    data_dict = {f"morphed2{i}": morph_coords2[:, i] for i in range(3)}
    data_dict.update({f"morphed{i}": morph_coords[:, i] for i in range(3)})
    data_dict.update({f"centered{i}": cent_coords[:, i] for i in range(3)})
    data_dict["fid"] = path.name

    coords_df.append(pd.DataFrame(data_dict))

coords_df = pd.concat(coords_df, ignore_index=1)

In [ ]:
valid_coords_df = coords_df[~np.isnan(coords_df["morphed0"])]

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(7, 3))
for i, k in enumerate(["morphed", "centered", "morphed2"]):
    axs[i].scatter(
        valid_coords_df[k + "1"],
        valid_coords_df[k + "2"],
        lw=0,
        s=5,
        alpha=0.03,
        color=(0.3,) * 3,
    )
    axs[i].set_title(k)
    axs[i].axis("equal")
    axs[i].axis("off")